In [1]:
!pip install seqval
!pip install --upgrade gensim

ERROR: Could not find a version that satisfies the requirement seqval (from versions: none)
ERROR: No matching distribution found for seqval
     |████████████████████████████████| 23.9MB 127kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
from google.colab import drive
from pathlib import Path

P = 'drive/MyDrive/knedle-data/NER_dataset'

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [4]:
!pip install seqeval

     |████████████████████████████████| 51kB 7.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=f9d19dd6cea4ef6543c74ca6810e96f0de3706f91920f9ef2a6f48a889380222
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [5]:
# Basic packages
import argparse
import numpy as np
import pandas as pd
# from gensim.models import KeyedVectors
import torch
from torch.utils.data import DataLoader
import itertools
import joblib
import re
import matplotlib.pyplot as plt
import operator
import math
from random import sample
# NER open packages
from seqeval.scheme import IOBES
from seqeval.metrics import f1_score
# my NER packages
from data3 import active_dataset
from utils import create_char2idx_dict, create_tag2idx_dict, create_word2idx_dict, new_custom_collate_fn, budget_limit, find_iobes_entities, find_iobes_entities2
from metrics import exact_f1_score, preprocess_pred_targ, IOBES_tags
from CNN_biLSTM_CRF import cnn_bilstm_crf
from CNN_CNN_LSTM2 import CNN_CNN_LSTM

import utils


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:

DATA_PATH=Path('drive/MyDrive/knedle_data/NER_dataset')
SAVE_PATH=Path('drive/MyDrive/knedle_data/NER_results')

args = dict(
    save_path=SAVE_PATH,
    epochs=50,
    dataset='aposentadoria',
    model='CNN-CNN-LSTM',
    lstm_hidden_size=128,
    batch_size=16,
    use_dev_set=False
)

parser = argparse.ArgumentParser()
parser.add_argument(
    '--save_path', 
    action='store', dest='save_path', 
    default=SAVE_PATH, type=str,
    # default='experiments/Supervised/', type=str
    )
parser.add_argument(
    '--epochs', 
    action='store', 
    dest='epochs', default=50, type=int)
parser.add_argument(
    '--dataset', 
    action='store', 
    dest='dataset', default='aposentadoria', type=str)
parser.add_argument(
    '--model', 
    action='store', 
    dest='model', default = 'CNN-CNN-LSTM', type=str)
parser.add_argument(
    '--lstm_hidden_size', 
    action='store', 
    dest='lstm_hidden_size', default=128, type=int)
parser.add_argument(
    '--batch_size', 
    action='store', 
    dest='batch_size', default=16, type=int)
parser.add_argument(
    '--use_dev_set', 
    action = 'store', 
    dest='use_dev_set', default=False, type=bool)
# parser_opt = parser.parse_args()
parser_opt, unknown = parser.parse_known_args()
print(f'Experiment:')


Experiment:


In [7]:
from importlib import reload as rl
import utils
rl(utils)

if parser_opt.dataset not in ['ontonotes', 'conll', 'aposentadoria']:
    raise ValueError(
        'Dataset not recognized. Options are: conll, ontonotes and aposentadoria'
    )

emb, train_path, test_path, data_format = utils.load_embedding(parser_opt, DATA_PATH)


In [9]:
rl(utils)

collate_object = utils.new_custom_collate_fn(
    pad_idx=emb.key_to_index['<PAD>'], 
    unk_idx=emb.key_to_index['<UNK>']
    # pad_idx=emb.vocab['<PAD>'].index, 
    # unk_idx=emb.vocab['<UNK>'].index
)

print('\nGenerating text2idx dictionaries (word, char, tag)')
word2idx = utils.create_word2idx_dict(emb, train_path)
char2idx = utils.create_char2idx_dict(train_path=train_path)
tag2idx  = utils.create_tag2idx_dict(train_path=train_path)

print('\nCreating training dataset')
train_set = active_dataset(
    data=train_path, 
    word2idx_dic=word2idx, 
    char2idx_dic=char2idx, tag2idx_dic=tag2idx, data_format=data_format)
# Putting all sentences into the labeled set for training
train_set.flag_labeled = False
train_set.label_data([i for i in range(len(train_set))])
train_set.flag_labeled = True

print('\nCreating test dataset')
test_set  = active_dataset(
    data=test_path, 
    word2idx_dic=word2idx, 
    char2idx_dic=char2idx, tag2idx_dic=tag2idx, data_format=data_format)
# Putting all sentences into the labeled set for testing
test_set.flag_labeled = False
test_set.label_data([i for i in range(len(test_set))])
test_set.flag_labeled = True
test_dataloader = DataLoader(test_set, batch_size=128, shuffle=False, collate_fn=collate_object)



Generating text2idx dictionaries (word, char, tag)
train_path: drive/MyDrive/knedle_data/NER_dataset/aposentadoria/aposentadoria_train.txt

Creating training dataset

Creating test dataset


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_opt = parser_opt.model
                     
# CNN-CNN-LSTM (greedy decoding) model
if model_opt == 'CNN-CNN-LSTM':
    model = CNN_CNN_LSTM(char_vocab_size=len(char2idx),
                                char_embedding_dim=25,
                                char_out_channels=50,
                                pretrained_word_emb=emb,
                                word2idx = word2idx,
                                word_out_channels=400,
                                word_conv_layers = 1,
                                num_classes=len(tag2idx),
                                decoder_layers = 1,
                                decoder_hidden_size = 128,
                                device=device)
    lrate = 0.010
    new_lrate = 0.010
    momentum = 0.9
    clipping_value = 5.0
    flag_adjust_lrate = False
    
elif model_opt == 'CNN-biLSTM-CRF':
    model = cnn_bilstm_crf(char_vocab_size=len(char2idx), 
                   char_embedding_dim=30, 
                   char_out_channels=30, 
                   pretrained_word_emb=emb, 
                   num_classes=len(tag2idx), 
                   device=device, 
                   lstm_hidden_size=parser_opt.lstm_hidden_size)
    lrate = 0.0025
    clipping_value = 5.0
    momentum = 0.9
    flag_adjust_lrate = False

model.to(device)


CNN_CNN_LSTM(
  (char_encoder): char_cnn(
    (embedding): Embedding(65, 25, padding_idx=0)
    (conv): Conv1d(25, 50, kernel_size=(3,), stride=(1,), padding=(1,))
    (relu): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (word_encoder): word_cnn(
    (embedding): Embedding(117, 50)
    (dropout): Dropout(p=0.5, inplace=False)
    (convnet): Sequential(
      (0): Conv1d(100, 400, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
    )
  )
  (decoder): decoder(
    (lstm): LSTM(542, 128)
    (linear): Linear(in_features=128, out_features=42, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (loss_fn): CrossEntropyLoss()
  )
)

In [11]:
supervised_epochs = parser_opt.epochs
optim = torch.optim.SGD(model.parameters(), lr=lrate, momentum=momentum)


# ==============================================================================================
# ==============================================================================================
# ============================= Supervised learning algorithm ==================================
# ==============================================================================================
# ==============================================================================================
print(f'\nInitiating supervised training\n\n')
f1_history = []

train_set.flag_labeled = True
batch_size = parser_opt.batch_size
dataloader = DataLoader(train_set, batch_size=batch_size, pin_memory=True, collate_fn = collate_object, shuffle=False)



Initiating supervised training




In [12]:
for epoch in range(10):
# for epoch in range(supervised_epochs):
    print(f'Epoch: {epoch}')
    model.train()        
    for sent, tag, word, mask in dataloader:
        sent = sent.to(device)
        tag = tag.to(device)
        word = word.to(device)
        mask = mask.to(device)
        optim.zero_grad()
        loss = model(sent, word, tag, mask)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clipping_value)
        optim.step()
    
    # Verify performance on test set after supervised training
    model.eval()
    with torch.no_grad():
        predictions, targets = preprocess_pred_targ(model, test_dataloader, device)
        predictions = IOBES_tags(predictions, tag2idx)
        targets = IOBES_tags(targets, tag2idx)
        micro_f1 = f1_score(targets, predictions, mode='strict', scheme=IOBES)
        f1_history.append(0 if np.isnan(micro_f1) else micro_f1)
        print(f'micro f1-score: {micro_f1}\n')


Epoch: 0
micro f1-score: 0.08015478164731896

Epoch: 1
micro f1-score: 0.5501415285078851

Epoch: 2
micro f1-score: 0.7710636838947458

Epoch: 3
micro f1-score: 0.8986074166797058

Epoch: 4
micro f1-score: 0.9639034472576954

Epoch: 5
micro f1-score: 0.9647163251719801

Epoch: 6
micro f1-score: 0.9731474688187822

Epoch: 7
micro f1-score: 0.9761505360659325

Epoch: 8
micro f1-score: 0.9849056603773584

Epoch: 9
micro f1-score: 0.9845596230518304



In [ ]:
hyperparams = {'model': str(model), 'LR': lrate, 'momentum': momentum, 'clipping': clipping_value}
dic = {'f1_hist': f1_history, 'hyperparams': hyperparams}
path = parser_opt.save_path.as_posix()

from glob import glob
cnt = 1
for name in glob(path+'/*.pkl'):
    if model_opt in name:
        cnt += 1
f_name = model_opt + '_' + str(cnt) + '.pkl'

joblib.dump(dic, path + f_name)
print(f'Training saved in: {path + f_name}')

Training saved in: drive/MyDrive/knedle_data/NER_resultsCNN-CNN-LSTM_1.pkl


# New Section

In [ ]:
!ls drive/MyDrive/knedle_data/NER_dataset

aposentadoria  conll03	README.md


In [ ]:
import torch
from torch import nn
from math import sqrt
# from torchcrf import CRF
# from drive.MyDrive.NER_code.crf import CRF
from crf import CRF
from torch.nn.utils.rnn import pad_sequence

class char_cnn(nn.Module):
    """single layer CNN with: filters=50, kernel_size=3, dropout=0.5
    CHANGES:
    - kaiming_uniform initialization of convolution weights
    - Embedding dropout with p=0.25
    - dropout now only applied on output of convolution layers (after activation function)
    - Added weight_norm to conv layers
    """
    def __init__(self, embedding_size, embedding_dim, char_out_channels):
        super(char_cnn, self).__init__()
        self.embedding = nn.Embedding(
            num_embeddings=embedding_size, 
            embedding_dim=embedding_dim, 
            padding_idx=0)
        self.conv = nn.Conv1d(
            in_channels=embedding_dim, 
            out_channels=char_out_channels,
            kernel_size=3, 
            stride=1, 
            padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.emb_dropout = nn.Dropout(p=0.5)
        self.init_weight()

    def init_weight(self):
        bias = sqrt(3/self.embedding.embedding_dim)
        nn.init.uniform_(self.embedding.weight, -bias, bias)
        # nn.init.kaiming_uniform_(self.conv.weight.data, mode='fan_in', nonlinearity='relu')

    def forward(self, x):
        x = self.emb_dropout(self.embedding(x))
        shape = x.shape
        x = self.conv(
            x.reshape([shape[0]*shape[1], shape[2], shape[3]]).permute(0, 2, 1)
        )
        # x = self.relu(x)
        # x = self.dropout(self.relu(x))
        x = torch.nn.functional.max_pool1d(x, kernel_size=x.shape[2]).squeeze(2)
        return x.reshape([shape[0], shape[1], -1])

In [ ]:

class bilstm_crf(nn.Module):
    def __init__(self, feature_size, num_classes, device, lstm_hidden_size=256):
        super(bilstm_crf, self).__init__()
        self.bilstm = torch.nn.LSTM(
            input_size=feature_size, 
            hidden_size=lstm_hidden_size, 
            num_layers=1, 
            batch_first=True, 
            bidirectional=True)
        self.linear = torch.nn.Linear(
            in_features=lstm_hidden_size*2, 
            out_features=num_classes)
        self.crf = CRF(num_tags=num_classes, batch_first=True)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.weight_init()

    def weight_init(self):
        # Initialize linear layer
        bias = sqrt(6/(self.linear.weight.shape[0]+self.linear.weight.shape[1]))
        nn.init.uniform_(self.linear.weight, -bias, bias)
        nn.init.constant_(self.linear.bias, 0.0)
        # Initialize LSTM layer
        for name, params in self.bilstm.named_parameters():
            if 'bias' in name:
                nn.init.constant_(params, 0.0)
                nn.init.constant_(params[self.bilstm.hidden_size:2*self.bilstm.hidden_size], 1.0)
            else:
                bias = sqrt(6/(params.shape[0]+params.shape[1]))
                nn.init.uniform_(params, -bias, bias)
        

    def forward(self, x, y, mask):
        x = self.dropout(x)
        x, (_, _) = self.bilstm(x)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.crf(x, y, mask=mask)
        return x

    def decode(self, x, mask):
        x, (_, _) = self.bilstm(x)
        x = self.linear(x)
        pred, prob = self.crf.decode(x, mask=mask)
        return pred, prob

In [ ]:

class cnn_bilstm_crf(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, char_out_channels, pretrained_word_emb, num_classes, device, lstm_hidden_size):
        super(cnn_bilstm_crf, self).__init__()
        self.num_classes = num_classes
        self.device = device
        self.char_encoder = char_cnn(
            embedding_size=char_vocab_size, 
            embedding_dim=char_embedding_dim, 
            char_out_channels=char_out_channels)
        self.word_embedder= nn.Embedding.from_pretrained(torch.FloatTensor(pretrained_word_emb.vectors))
        self.decoder      = bilstm_crf(
            feature_size=char_out_channels+pretrained_word_emb.vector_size, 
            num_classes=num_classes, 
            device=device, 
            lstm_hidden_size=lstm_hidden_size)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, sent, word, tag, mask):
        char_emb = self.char_encoder(word)
        word_emb = self.word_embedder(sent)
        x = torch.cat((word_emb, char_emb), dim=2)
        x = self.decoder(x, tag, mask)
        return -x

    def decode(self, sent, word, mask, return_token_log_probabilities = False):
        """
        return_token_log_probabilities not implemented
        """
        char_emb = self.char_encoder(word)
        word_emb = self.word_embedder(sent)
        x = torch.cat((word_emb, char_emb), dim=2)
        x, prob = self.decoder.decode(x, mask=mask)
        x = [torch.LongTensor(aux) for aux in x]
        predictions = pad_sequence(x, batch_first = True, padding_value = 0)
        return predictions, prob
